In [1]:
import sys
import numpy as np
from copy import deepcopy

sys.path.append("./src/")
from pdb_numpy import Coor, Model

In [5]:

test = Coor('src/pdb_numpy/tests/input/2ol9.cif')
print(test.crystal_pack)
print(test.data_mmCIF)

test.write("test.pdb", check_file_out=False)



OrderedDict([('title', OrderedDict([('title', 'data_2OL9')])), ('_entry', OrderedDict([('id', '2OL9')])), ('_audit_conform', OrderedDict([('dict_name', 'mmcif_pdbx.dic'), ('dict_version', '5.289'), ('dict_location', 'http://mmcif.pdb.org/dictionaries/ascii/mmcif_pdbx.dic')])), ('_database_2', {'col_names': ['database_id', 'database_code'], 'value': [['PDB', 'RCSB', 'WWPDB'], ['2OL9', 'RCSB041276', 'D_1000041276']]}), ('_pdbx_database_related', OrderedDict([('db_name', 'PDB'), ('db_id', '2OKZ'), ('details', '.'), ('content_type', 'unspecified')])), ('_pdbx_database_status', OrderedDict([('entry_id', '2OL9'), ('deposit_site', 'RCSB'), ('process_site', 'RCSB'), ('recvd_initial_deposition_date', '2007-01-18'), ('status_code', 'REL'), ('status_code_sf', 'REL'), ('status_code_mr', '?'), ('SG_entry', '?'), ('pdb_format_compatible', 'Y'), ('status_code_cs', '?'), ('methods_development_category', '?')])), ('_audit_author', {'col_names': ['name', 'pdbx_ordinal'], 'value': [["'Apostol, M.A.'", "

In [14]:
def pdb_parse_transformation(text):
    """Parse the `REMARK 350   BIOMT` information from a pdb file.

    Parameters
    ----------
    text : str
        pdb file

    Returns
    -------
    symetry_dict : dict
        symetry information
    """

    transformation_dict = {}

    for line in text.split("\n"):
        if line[11:23] == "BIOMOLECULE:":
            biomol = int(line[24:])
            transformation_dict[biomol] = {"chains": [], "matrix": []}
        elif line[34:41] == "CHAINS:":
            transformation_dict[biomol]["chains"] += line[42:].split()
        elif line.startswith("REMARK 350   BIOMT"):
            transformation_dict[biomol]["matrix"] += [
                [float(x) for x in line[19:].split()]
            ]

    return transformation_dict

txt = """REMARK 350                                                                      
REMARK 350 COORDINATES FOR A COMPLETE MULTIMER REPRESENTING THE KNOWN           
REMARK 350 BIOLOGICALLY SIGNIFICANT OLIGOMERIZATION STATE OF THE                
REMARK 350 MOLECULE CAN BE GENERATED BY APPLYING BIOMT TRANSFORMATIONS          
REMARK 350 GIVEN BELOW.  BOTH NON-CRYSTALLOGRAPHIC AND                          
REMARK 350 CRYSTALLOGRAPHIC OPERATIONS ARE GIVEN.                               
REMARK 350                                                                      
REMARK 350 BIOMOLECULE: 1                                                       
REMARK 350 AUTHOR DETERMINED BIOLOGICAL UNIT: DIMERIC                           
REMARK 350 APPLY THE FOLLOWING TO CHAINS: A                                     
REMARK 350   BIOMT1   1  1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   1  0.000000  1.000000  0.000000        0.00000            
REMARK 350   BIOMT3   1  0.000000  0.000000  1.000000        0.00000            
REMARK 350   BIOMT1   2  1.000000  0.000000  0.000000        0.94015            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        4.78756            
REMARK 350   BIOMT3   2  0.000000  0.000000  1.000000        0.00000"""

pdb_parse_transformation(txt)

{1: {'chains': ['A'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0],
   [2.0, 1.0, 0.0, 0.0, 0.94015],
   [2.0, 0.0, 1.0, 0.0, 4.78756],
   [2.0, 0.0, 0.0, 1.0, 0.0]]}}

In [16]:

def parse_transformation(data_mmCIF):
    """Parse the `_pdbx_struct_oper_list` information from a mmcif file.

    Parameters
    ----------
    pdbx_struct_oper_list : dict
        mmcif file information

    Returns
    -------
    symetry_dict : dict
        symetry information
    """

    print(data_mmCIF['_pdbx_struct_assembly'])
    print(data_mmCIF['_pdbx_struct_assembly_gen'])
    print(data_mmCIF['_pdbx_struct_oper_list'])

    transformation_dict = {}

    return transformation_dict


In [17]:
parse_transformation(test.data_mmCIF)

OrderedDict([('id', '1'), ('details', 'author_defined_assembly'), ('method_details', '?'), ('oligomeric_details', 'dimeric'), ('oligomeric_count', '2')])
OrderedDict([('assembly_id', '1'), ('oper_expression', '1,2'), ('asym_id_list', 'A,B')])
{'col_names': ['id', 'type', 'name', 'symmetry_operation', 'matrix[1][1]', 'matrix[1][2]', 'matrix[1][3]', 'vector[1]', 'matrix[2][1]', 'matrix[2][2]', 'matrix[2][3]', 'vector[2]', 'matrix[3][1]', 'matrix[3][2]', 'matrix[3][3]', 'vector[3]'], 'value': [['1', '2'], ["'identity operation'", "'crystal symmetry operation'"], ['1_555', '1_565'], ['x,y,z', 'x,y+1,z'], ['1.0000000000', '1.0000000000'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '0.9401502764'], ['0.0000000000', '0.0000000000'], ['1.0000000000', '1.0000000000'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '4.7875628934'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '0.0000000000'], ['1.0000000000', '1.0000000000'], ['0.0000000000', '

{}

In [ ]:

assert test.len == 56
# Here there is only identity matrix
# as symmetry, so no change is expected
test.add_symmetry()
assert test.len == 56
test.apply_transformation()
assert test.len == 112
test.copy_box(3, 3, 3)
assert test.len == 3024
test.compute_chains_CA()
test.remove_overlap_chain()
assert test.len == 3024

test.write("test.pdb", check_file_out=False)


In [3]:
test.write('tmp.pdb', check_file_out=False)

In [4]:
test_2 = Coor('tmp.pdb')
test_2.write('tmp2.gro', check_file_out=False)

In [4]:
test.add_symmetry()
test.apply_transformation(index=1)
test.compute_chains_CA()
test.write_pdb('tmp.pdb', check_file_out=False)

Add symmetry 1
0
1
Add transformation 1
2
Add transformation 2


In [5]:
test.symmetry

{'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 0.0, 1.0, 0.0],
  [2.0, -1.0, 0.0, 0.0, 0.0],
  [2.0, 0.0, 1.0, 0.0, 2.35],
  [2.0, 0.0, 0.0, -1.0, 0.0]]}

In [8]:
test.models

In [6]:
print(test.transformation)

{1: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 4.7], [2.0, 0.0, 0.0, 1.0, 0.0], [3.0, 1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -4.7], [3.0, 0.0, 0.0, 1.0, 0.0]]}, 2: {'chains': ['A'], 'matrix': [[1.0, -1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 2.35], [1.0, 0.0, 0.0, -1.0, 0.0], [2.0, -1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 7.05], [2.0, 0.0, 0.0, -1.0, 0.0], [3.0, -1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -2.35], [3.0, 0.0, 0.0, -1.0, 0.0]]}, 3: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0]]}}


In [22]:
def parse_transformation(text):

    transformation_dict = {}

    for line in text.split('\n'):
        if line[11:23] == "BIOMOLECULE:":
            biomol = int(line[24:])
            transformation_dict[biomol] = {'chains': [], 'matrix': []}
        elif line[34:41] == "CHAINS:":
            transformation_dict[biomol]['chains'] += line[42:].split()
        elif line.startswith('REMARK 350   BIOMT'):
            transformation_dict[biomol]['matrix'] += [[float(x) for x in line[19:].split()]]
        #print(line, line[34:])
    print(transformation_dict)


parse_transformation(test.transformation)

{1: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 4.7], [2.0, 0.0, 0.0, 1.0, 0.0], [3.0, 1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -4.7], [3.0, 0.0, 0.0, 1.0, 0.0]]}, 2: {'chains': ['A'], 'matrix': [[1.0, -1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 2.35], [1.0, 0.0, 0.0, -1.0, 0.0], [2.0, -1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 7.05], [2.0, 0.0, 0.0, -1.0, 0.0], [3.0, -1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -2.35], [3.0, 0.0, 0.0, -1.0, 0.0]]}, 3: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0]]}}


In [34]:
test.models.append(deepcopy(test.models[0]))
test.models.append(deepcopy(test.models[0]))

In [4]:
text = \
"""REMARK 350   BIOMT1   2 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        7.05000            
REMARK 350   BIOMT3   2  0.000000  0.000000 -1.000000        0.00000"""

rot_mat = []

for line in text.split("\n"):
    rot_mat.append([float(x) for x in line.split()[4:]])

rot_mat = np.array(rot_mat, dtype=object)
print(rot_mat)

text = \
"""REMARK 350   BIOMT1   3 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   3  0.000000  1.000000  0.000000       -2.35000            
REMARK 350   BIOMT3   3  0.000000  0.000000 -1.000000        0.00000"""

rot_mat_2 = []

for line in text.split("\n"):
    rot_mat_2.append([float(x) for x in line.split()[4:]])

rot_mat_2 = np.array(rot_mat_2, dtype=object)
print(rot_mat_2)

[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 7.05]
 [0.0 0.0 -1.0 0.0]]
[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 -2.35]
 [0.0 0.0 -1.0 0.0]]


In [21]:
rot_mat[:,:3]

array([[-1.0, 0.0, 0.0],
       [0.0, 1.0, 0.0],
       [0.0, 0.0, -1.0]], dtype=object)

In [36]:
test.models[1].xyz = np.dot(test.models[0].xyz, rot_mat[:,:3]) + rot_mat[:,3]
test.models[2].xyz = np.dot(test.models[0].xyz, rot_mat_2[:,:3]) + rot_mat_2[:,3]

In [37]:
test.merge_model()
test.write_pdb('tmp.pdb', check_file_out=False)

In [25]:
pwd

'/home/murail/Documents/Code/pdb_numpy'